In [1]:
!pip3 install pandas
!pip3 install numpy
!pip3 install seaborn
!pip3 install matplotlib
!pip3 install statsmodels
!pip3 install sklearn
!pip3 install Tokenizer
!pip3 install nltk
!pip install Prophet

In [2]:
!pip install scikit-learn
!pip install plotly

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import statsmodels.api as sm
import seaborn as sns
from tensorflow.keras.preprocessing.text import Tokenizer
import tensorflow as tf
from sklearn.metrics import mean_squared_error
from prophet import Prophet

2023-11-19 17:38:10.549086: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
df = pd.read_pickle('shared/Project-3_NYC_311_Calls.pkl')

In [5]:
df.columns

Index(['Unique Key', 'Created Date', 'Agency', 'Agency Name', 'Complaint Type',
       'Descriptor', 'Location Type', 'Incident Zip', 'City',
       'Resolution Description', 'Borough', 'Open Data Channel Type'],
      dtype='object')

In [6]:
df.index = pd.DatetimeIndex(df['Created Date'])

In [7]:
df.drop(columns='Created Date', inplace=True)

In [8]:
df.head()

,Unique Key,Agency,Agency Name,Complaint Type,Descriptor,Location Type,Incident Zip,City,Resolution Description,Borough,Open Data Channel Type
Created Date,,,,,,,,,,,
2011-04-06 00:00:00,20184537,HPD,Department of Housing Preservation and Develop...,HEATING,HEAT,RESIDENTIAL BUILDING,10002.0,NEW YORK,More than one complaint was received for this ...,MANHATTAN,UNKNOWN
2011-04-06 00:00:00,20184538,HPD,Department of Housing Preservation and Develop...,GENERAL CONSTRUCTION,WINDOWS,RESIDENTIAL BUILDING,11236.0,BROOKLYN,The Department of Housing Preservation and Dev...,BROOKLYN,UNKNOWN
2011-04-06 00:00:00,20184539,HPD,Department of Housing Preservation and Develop...,PAINT - PLASTER,WALLS,RESIDENTIAL BUILDING,10460.0,BRONX,The Department of Housing Preservation and Dev...,BRONX,UNKNOWN
2022-07-08 11:14:43,54732265,DSNY,Department of Sanitation,Dirty Condition,Trash,Sidewalk,10467.0,BRONX,The Department of Sanitation investigated this...,BRONX,PHONE
2011-04-06 00:00:00,20184540,HPD,Department of Housing Preservation and Develop...,NONCONST,VERMIN,RESIDENTIAL BUILDING,10460.0,BRONX,The Department of Housing Preservation and Dev...,BRONX,UNKNOWN


In [9]:
df.shape

(33780977, 11)

In [10]:
print(df.index.max())
print(df.index.min())

2023-08-04 12:00:00
2010-01-01 00:00:00


In [11]:
df_subset = df['Unique Key']

In [12]:
df_subset.dropna()

Created Date
2011-04-06 00:00:00    20184537
2011-04-06 00:00:00    20184538
2011-04-06 00:00:00    20184539
2022-07-08 11:14:43    54732265
2011-04-06 00:00:00    20184540
                         ...   
2011-04-06 00:00:00    20184532
2011-04-06 00:00:00    20184533
2011-04-06 00:00:00    20184534
2011-04-06 00:00:00    20184535
2011-04-06 00:00:00    20184536
Name: Unique Key, Length: 33780977, dtype: int64

In [13]:
df_daily = df_subset.resample('D').count()
average_daily_complaints_2022 = df_daily[df_daily.index.year == 2022].mean()
print(average_daily_complaints_2022)

8684.320547945206


In [14]:
max_calls_date = df_daily.idxmax()

max_calls_date

Timestamp('2020-08-04 00:00:00')

In [15]:
data_max_date = df.loc[max_calls_date.strftime('2020-08-04')]
top_complaint = data_max_date['Complaint Type'].value_counts().idxmax()
top_complaint

'Damaged Tree'

In [16]:
monthly_call_counts = df['Unique Key'].resample('M').count()

In [17]:
from statsmodels.tsa.seasonal import seasonal_decompose

In [18]:
quietest_month = monthly_call_counts.idxmin()
quietest_month

Timestamp('2023-08-31 00:00:00')

In [19]:
quietest_month_name = quietest_month.strftime('%B %Y')
quietest_calls = monthly_call_counts.min()
quietest_month_name

'August 2023'

In [20]:
result = seasonal_decompose(df_daily, model='additive')
seasonal = result.seasonal['2020-12-25']
seasonal

182.69763790386236

In [21]:
autocorrelation = df_daily.autocorr(lag=1)
autocorrelation

0.7517059728398577

In [23]:
from prophet import Prophet
from sklearn.metrics import mean_squared_error
import numpy as np

prophet_df = df_daily.reset_index()
prophet_df.columns = ['ds', 'y']

prophet_df['ds'] = pd.to_datetime(prophet_df['ds'])

train = prophet_df.iloc[:-90]
test = prophet_df.iloc[-90:]

model = Prophet()
model.fit(train)

future = model.make_future_dataframe(periods=90)
forecast = model.predict(future)

test_forecast = forecast.set_index('ds').loc[test['ds']]
rmse = np.sqrt(mean_squared_error(test['y'], test_forecast['yhat']))
print(f"RMSE on test set: {rmse}")

17:46:08 - cmdstanpy - INFO - Chain [1] start processing
17:46:09 - cmdstanpy - INFO - Chain [1] done processing


RMSE on test set: 1233.7823321393885
